# LightGBoost

## Import

In [19]:
# Preprocessing
import pandas as pd
import numpy as np
from dask import dataframe as dd 
from sklearn.model_selection import train_test_split

# Import de modèle
import lightgbm as lgb
from dask_ml.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

# Connection cluster
from sklearn.externals import joblib
from dask.distributed import Client
try:
    client1.close()
except:
    print(' pas de client')
#client1 = Client('c126-23.enst.fr:8786')  # set up local cluster on your laptop
#client1=Client()
#client1


## Normalisation du dataset et sauvegarde de celui-ci

In [2]:
xtrain = pd.read_csv('xtrain_challenge.csv')
ytrain = pd.read_csv('ytrain_challenge.csv')
xtest = pd.read_csv('xtest_challenge.csv')
xtest['y']=1
dataset=xtrain.merge(ytrain,  left_index=True,right_index=True)

In [3]:
dfdd = dd.from_pandas(dataset, chunksize=10000)
xtestdd =dd.from_pandas(xtest, chunksize=10000)
dfdd.persist()
xtestdd.persist()

,fA1,fA2,fA3,fA4,fA5,fA6,fA7,fA8,fA9,fA10,fA11,fA12,fA13,fA14,fB1,fB2,fB3,fB4,fB5,fB6,fB7,fB8,fB9,fB10,fB11,fB12,fB13,fB14,s1,s2,s3,s4,s5,s6,s7,s8,y
npartitions=160,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64
10000,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1590000,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1598218,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [4]:
scaler = StandardScaler()

In [5]:
scaler.fit(dfdd,dfdd.y)
scaler.transform(dfdd,dfdd.y)
scaler.transform(xtestdd,xtestdd.y)

,fA1,fA2,fA3,fA4,fA5,fA6,fA7,fA8,fA9,fA10,fA11,fA12,fA13,fA14,fB1,fB2,fB3,fB4,fB5,fB6,fB7,fB8,fB9,fB10,fB11,fB12,fB13,fB14,s1,s2,s3,s4,s5,s6,s7,s8,y
npartitions=160,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
10000,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1590000,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1598218,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [6]:
des=dfdd.describe()
des.compute()

,fA1,fA2,fA3,fA4,fA5,fA6,fA7,fA8,fA9,fA10,...,fB14,s1,s2,s3,s4,s5,s6,s7,s8,y
count,3.196465e+06,3.196465e+06,3.196465e+06,3.196465e+06,3.196465e+06,3.196465e+06,3.196465e+06,3.196465e+06,3.196465e+06,3.196465e+06,...,3.196465e+06,3.196465e+06,3.196465e+06,3.196465e+06,3.196465e+06,3.196465e+06,3.196465e+06,3.196465e+06,3.196465e+06,3.196465e+06
mean,3.123656e+00,7.200140e-01,8.895844e-02,2.938227e+01,9.112188e-01,4.165448e-03,8.456615e-02,1.323673e-01,2.970159e-02,-2.833317e-04,...,2.477718e+02,3.874350e+03,2.434998e+03,4.200151e+03,4.242067e+03,3.488791e+03,3.399066e+03,3.509614e+03,3.745971e+03,2.857347e-01
std,1.618458e+00,4.224322e-01,2.824698e-01,9.793075e+00,2.757797e-01,4.433567e-02,2.729676e-01,4.159110e-01,1.279389e-01,2.096133e-02,...,1.221232e+02,2.063822e+03,2.272729e+02,2.954367e+03,2.127659e+03,1.360081e+03,1.363354e+03,1.066627e+03,1.548290e+03,4.517637e-01
min,-4.500000e-01,0.000000e+00,0.000000e+00,1.800000e+01,0.000000e+00,0.000000e+00,0.000000e+00,-2.550000e+00,-3.030000e+00,-3.000000e-01,...,-3.087900e+02,1.212700e+03,1.511400e+03,7.218000e+02,1.133800e+03,6.682000e+02,7.200000e+02,1.358000e+03,7.107000e+02,0.000000e+00
25%,2.070000e+00,3.000000e-01,0.000000e+00,2.200000e+01,1.000000e+00,0.000000e+00,0.000000e+00,-2.000000e-02,-2.000000e-02,0.000000e+00,...,2.260450e+02,2.445775e+03,2.304000e+03,2.123775e+03,2.757175e+03,2.481650e+03,2.359950e+03,2.763925e+03,2.645700e+03,0.000000e+00
50%,3.450000e+00,1.000000e+00,0.000000e+00,2.600000e+01,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e-02,2.000000e-02,0.000000e+00,...,2.799300e+02,3.107100e+03,2.448350e+03,3.144500e+03,3.451200e+03,3.098400e+03,3.044650e+03,3.178200e+03,3.262500e+03,0.000000e+00
75%,4.440000e+00,1.000000e+00,0.000000e+00,3.500000e+01,1.000000e+00,0.000000e+00,0.000000e+00,1.100000e-01,8.000000e-02,1.000000e-02,...,3.236600e+02,4.725775e+03,2.598325e+03,5.362500e+03,5.232525e+03,4.542625e+03,4.469825e+03,4.054500e+03,4.624425e+03,1.000000e+00
max,7.940000e+00,1.000000e+00,1.000000e+00,7.400000e+01,1.000000e+00,1.000000e+00,1.000000e+00,2.500000e+00,7.600000e-01,2.600000e-01,...,4.814200e+02,1.204420e+04,3.035900e+03,1.666650e+04,1.181290e+04,7.731000e+03,7.580100e+03,6.949500e+03,8.524900e+03,1.000000e+00


In [7]:
dfpd=dfdd.compute()
xtestpd=xtestdd.compute()
dfpd.describe()

,fA1,fA2,fA3,fA4,fA5,fA6,fA7,fA8,fA9,fA10,...,fB14,s1,s2,s3,s4,s5,s6,s7,s8,y
count,3.196465e+06,3.196465e+06,3.196465e+06,3.196465e+06,3.196465e+06,3.196465e+06,3.196465e+06,3.196465e+06,3.196465e+06,3.196465e+06,...,3.196465e+06,3.196465e+06,3.196465e+06,3.196465e+06,3.196465e+06,3.196465e+06,3.196465e+06,3.196465e+06,3.196465e+06,3.196465e+06
mean,3.123656e+00,7.200140e-01,8.895844e-02,2.938227e+01,9.112188e-01,4.165448e-03,8.456615e-02,1.323673e-01,2.970159e-02,-2.833317e-04,...,2.477718e+02,3.874350e+03,2.434998e+03,4.200151e+03,4.242067e+03,3.488791e+03,3.399066e+03,3.509614e+03,3.745971e+03,2.857347e-01
std,1.618458e+00,4.224322e-01,2.824698e-01,9.793075e+00,2.757797e-01,4.433567e-02,2.729676e-01,4.159110e-01,1.279389e-01,2.096133e-02,...,1.221232e+02,2.063822e+03,2.272729e+02,2.954367e+03,2.127659e+03,1.360081e+03,1.363354e+03,1.066627e+03,1.548290e+03,4.517637e-01
min,-4.500000e-01,0.000000e+00,0.000000e+00,1.800000e+01,0.000000e+00,0.000000e+00,0.000000e+00,-2.550000e+00,-3.030000e+00,-3.000000e-01,...,-3.087900e+02,1.212700e+03,1.511400e+03,7.218000e+02,1.133800e+03,6.682000e+02,7.200000e+02,1.358000e+03,7.107000e+02,0.000000e+00
25%,1.950000e+00,2.500000e-01,0.000000e+00,2.200000e+01,1.000000e+00,0.000000e+00,0.000000e+00,-2.000000e-02,-2.000000e-02,0.000000e+00,...,2.214200e+02,2.420300e+03,2.296300e+03,2.086200e+03,2.735400e+03,2.457200e+03,2.335000e+03,2.742600e+03,2.620100e+03,0.000000e+00
50%,3.390000e+00,1.000000e+00,0.000000e+00,2.600000e+01,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e-02,0.000000e+00,...,2.766400e+02,3.062600e+03,2.439800e+03,3.073300e+03,3.402000e+03,3.063300e+03,2.999800e+03,3.151800e+03,3.218800e+03,0.000000e+00
75%,4.380000e+00,1.000000e+00,0.000000e+00,3.400000e+01,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e-01,8.000000e-02,1.000000e-02,...,3.216600e+02,4.442500e+03,2.589300e+03,5.143800e+03,4.898800e+03,4.406800e+03,4.375500e+03,3.926300e+03,4.396900e+03,1.000000e+00
max,7.940000e+00,1.000000e+00,1.000000e+00,7.400000e+01,1.000000e+00,1.000000e+00,1.000000e+00,2.500000e+00,7.600000e-01,2.600000e-01,...,4.814200e+02,1.204420e+04,3.035900e+03,1.666650e+04,1.181290e+04,7.731000e+03,7.580100e+03,6.949500e+03,8.524900e+03,1.000000e+00


In [8]:
dfpd.to_csv('dataNormalized.csv')
xtestpd.drop('y',axis=1).to_csv('testNormalized.csv')

## Calcul du modèle

In [9]:
data=pd.read_csv('dataNormalized.csv',index_col=0)
train,test=train_test_split(data,test_size=0.1,stratify=data['y'])

/anaconda3/envs/python36/lib/python3.6/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [10]:
bst =  lgb.LGBMClassifier(learning_rate=0.02,
                        n_estimators=500,
                        n_jobs=-1)

paramGrid_lat = {
        'num_leaves':[32,64,128,256],
        'learning_rate':[0.02,0.4,0.7]
    }

gsc=GridSearchCV(bst,param_grid=paramGrid_lat,iid=False,cv=2)

In [11]:
#with joblib.parallel_backend("dask",scatter=[train.drop('y',axis=1),train.y]):
#    print('start')
gsc.fit(train.drop('y',axis=1),train.y)

GridSearchCV(cv=2, error_score='raise-deprecating',
       estimator=LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.02, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=500, n_jobs=-1, num_leaves=31, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0),
       fit_params=None, iid=False, n_jobs=None,
       param_grid={'num_leaves': [32, 64, 128, 256], 'learning_rate': [0.02, 0.4, 0.7]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

## Sauvegarde des résultats et affichage

In [12]:
result=pd.DataFrame(gsc.cv_results_).sort_values('rank_test_score')
result.to_csv('LGBM.csv',index=None)
result

/anaconda3/envs/python36/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/anaconda3/envs/python36/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/anaconda3/envs/python36/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/anaconda3/envs/python36/li

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_num_leaves,params,split0_test_score,split1_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,mean_train_score,std_train_score
3,98.779547,0.946638,14.711493,0.277883,0.02,256,"{'learning_rate': 0.02, 'num_leaves': 256}",0.987912,0.987832,0.987872,0.000040,1,0.990968,0.991069,0.991019,0.000050
2,94.529286,0.926894,13.868437,0.247973,0.02,128,"{'learning_rate': 0.02, 'num_leaves': 128}",0.986516,0.986479,0.986498,0.000018,2,0.988014,0.988184,0.988099,0.000085
1,68.146726,0.511424,11.636487,0.263768,0.02,64,"{'learning_rate': 0.02, 'num_leaves': 64}",0.985063,0.984973,0.985018,0.000045,3,0.985731,0.985950,0.985840,0.000110
0,58.402427,4.384872,9.723119,0.349699,0.02,32,"{'learning_rate': 0.02, 'num_leaves': 32}",0.983464,0.983290,0.983377,0.000087,4,0.983764,0.983920,0.983842,0.000078
4,28.826629,0.630166,7.994866,0.201128,0.4,32,"{'learning_rate': 0.4, 'num_leaves': 32}",0.983425,0.983252,0.983339,0.000087,5,0.984965,0.984797,0.984881,0.000084
8,26.649774,0.406546,8.610677,0.083260,0.7,32,"{'learning_rate': 0.7, 'num_leaves': 32}",0.981956,0.981920,0.981938,0.000018,6,0.983401,0.983681,0.983541,0.000140
5,35.504586,0.599904,10.101779,0.123615,0.4,64,"{'learning_rate': 0.4, 'num_leaves': 64}",0.980784,0.981377,0.981080,0.000296,7,0.982666,0.983824,0.983245,0.000579
9,34.976297,0.386149,10.287855,0.128522,0.7,64,"{'learning_rate': 0.7, 'num_leaves': 64}",0.979862,0.980125,0.979993,0.000131,8,0.981306,0.981954,0.981630,0.000324
6,50.079604,1.605808,12.735095,0.204151,0.4,128,"{'learning_rate': 0.4, 'num_leaves': 128}",0.976314,0.979722,0.978018,0.001704,9,0.979315,0.982140,0.980728,0.001412
7,70.000215,1.332182,16.317688,0.151828,0.4,256,"{'learning_rate': 0.4, 'num_leaves': 256}",0.968426,0.970547,0.969486,0.001061,10,0.971408,0.973724,0.972566,0.001158


In [13]:
filename = 'LGBM_grid.sav'
joblib.dump( gsc, filename)

['LGBM_grid.sav']

In [14]:
yvalid = gsc.predict(test.drop('y',axis=1))
(yvalid == test['y']).mean()

0.9881994825541925

## Calcul du modèle final

In [15]:
gbm = lgb.LGBMClassifier(num_leaves=124,
                        learning_rate=0.02,
                        n_estimators=10000,
                        n_jobs=-1)
gbm.fit(train.drop('y',axis=1), train['y'],
        eval_set=[(test.drop('y',axis=1),  test['y'])],
        early_stopping_rounds=15,verbose=False)


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.02, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=10000, n_jobs=-1, num_leaves=124, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [16]:
yvalid = gbm.predict(test.drop('y',axis=1))
(yvalid == test['y']).mean()

0.9932832155471505

In [17]:
filename2 = 'LGBM_final.sav'
joblib.dump( gbm, filename2)

['LGBM_final.sav']

## Prepare a file for submission

In [18]:
# Load test data
xtest = pd.read_csv('testNormalized.csv',index_col=0)
model = joblib.load(filename2)
ytest = model.predict(xtest)
print(ytest.shape)
np.savetxt('ytest_gmb_final.csv', ytest, fmt = '%1.0d', delimiter=',')

/anaconda3/envs/python36/lib/python3.6/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


(1598219,)
